In [3]:
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import matplotlib.pyplot as plt
# from detecto.utils import read_image 
# from detecto.core import Dataset, Model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
train_dir = os.path.join('/kaggle/input/garbage-classification/Garbage classification/Garbage classification')
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [5]:
train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                                   rotation_range=15,zoom_range=0.1,
                                   width_shift_range=0.15,height_shift_range=0.15,
                                   shear_range=0.1,
                                   fill_mode="nearest",
                                   rescale=1./255., 
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(384, 512), batch_size=32, class_mode='binary', subset='training')
validation_generator = train_datagen.flow_from_directory(train_dir, target_size=(384, 512), batch_size=32, class_mode='binary', subset='validation')

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [6]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.90):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [7]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(384, 512, 3)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [8]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = 0.001), metrics=['accuracy'])

In [9]:
history = model.fit(train_generator, epochs=50, verbose=1, validation_data=validation_generator, callbacks=[callbacks])

Epoch 1/50


2023-05-12 09:52:36.878419: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


64/64 [==============================] - 175s 2s/step - loss: 1.9909 - accuracy: 0.4382 - val_loss: 3.0780 - val_accuracy: 0.1849
Epoch 2/50
64/64 [==============================] - 133s 2s/step - loss: 1.3540 - accuracy: 0.5380 - val_loss: 2.4756 - val_accuracy: 0.1869
Epoch 3/50
64/64 [==============================] - 133s 2s/step - loss: 1.2002 - accuracy: 0.5776 - val_loss: 1.9528 - val_accuracy: 0.2326
Epoch 4/50
64/64 [==============================] - 133s 2s/step - loss: 1.1239 - accuracy: 0.5914 - val_loss: 2.6400 - val_accuracy: 0.2406
Epoch 5/50
64/64 [==============================] - 132s 2s/step - loss: 1.0822 - accuracy: 0.6047 - val_loss: 2.7789 - val_accuracy: 0.2664
Epoch 6/50
64/64 [==============================] - 131s 2s/step - loss: 1.0105 - accuracy: 0.6240 - val_loss: 3.3303 - val_accuracy: 0.2247
Epoch 7/50
64/64 [==============================] - 131s 2s/step - loss: 1.0591 - accuracy: 0.6255 - val_loss: 4.2287 - val_accuracy: 0.2167
Epoch 8/50
64/64 [======

In [10]:
model.save('Litter Detection')

In [19]:
model.save('model1.h5')

In [15]:
import cv2
import torch
import numpy as np

class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

camera = cv2.VideoCapture(0)

def detect_objects(camera, class_names, confidence_threshold, model_path):
    model = torch.hub.load('custom', path=model_path)
    model.conf = confidence_threshold
    
    while True:

        ret, frame = camera.read()
        
        with torch.no_grad():
            results = model(frame)
    
        predictions = results.pandas().xyxy[0]
        predictions = predictions[predictions['confidence'] >= confidence_threshold]
        labels = predictions['class'].astype(int).tolist()
        boxes = predictions[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
        
        print(predictions)
        print(boxes)
        
        for label, box in zip(labels, boxes):
            if len(box) != 4:
                print(f"Invalid box: {box}")
                continue
            x_min, y_min, x_max, y_max = map(int, box)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, class_names[label], (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    camera.release()
    cv2.destroyAllWindows()

detect_objects(camera, class_names, confidence_threshold=0.5, model_path='/kaggle/working/model.pt')

[ WARN:0] global /tmp/pip-req-build-4x5kub8r/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


TypeError: load() missing 1 required positional argument: 'model'